In [ ]:
#5 систем
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import imageio.v2 as imageio
from io import BytesIO
from skimage.morphology import footprint_rectangle
from scipy.ndimage import convolve
from skimage.filters import threshold_local
from scipy.ndimage import binary_closing, binary_opening
from skimage.filters import gaussian, threshold_local
from scipy.ndimage import binary_closing, binary_opening

from scipy import ndimage
from skimage import morphology
from skimage.filters import threshold_otsu

class ReactionDiffusionModel:
    def __init__(self, with_animation=False):
        self.with_animation = with_animation  # Флаг для анимации

        # Выбор системы
        print("\nДоступные системы активатор-ингибитор:")
        print("1. Мейнхардта (Для задачи кальцификации стенок аортального клапана)")
        print("   Уравнения:")
        print("   ∂u/∂t = Du∇²u + γ(u²/((1 + ku²)v) - cu)")
        print("   ∂v/∂t = Dv∇²v + γ(u² - ev + S)")

        print("2. Шнакенберга")#
        print("   Уравнения:")
        print("   ∂u/∂t = Du∇²u + γ(a - u + u²v)")
        print("   ∂v/∂t = Dv∇²v + γ(b - u²v)")

        print("3. Система с кубическим членом в активаторе")
        print("   Уравнения:")
        print("   ∂u/∂t = Du∇²u + γ(a - u + u²v - u³)")
        print("   ∂v/∂t = Dv∇²v + γ(b - u²v)")

        print("4. Система с третьим полем Y")
        print("   Уравнения:")
        print("   ∂u/∂t = Du∇²u + ρ_a(u²v/(1 + K_a*u²) - u)")
        print("   ∂v/∂t = Dv∇²v + σ_s/(1 + K_s*y) - (ρ_s*u²v)/(1 + K_a*u²) - μ_s*v")
        print("   ∂y/∂t = Dy∇²y + ρ_y*(y²/(1 + K_y*y²)) - μ_y*y + σ_y*u")

        print("5. Система 'круговой области' ")
        print("   Уравнения:")
        print("   ∂u/∂t = Du∇²u +γ(u² - uv)")
        print("   ∂v/∂t = Dv∇²v + γ(1 + b² - (1 - a² - b²)v - (2 - a²)uv)")

        while True:
            try:
                system_choice = int(input("Выберите систему (1-5): "))
                if 1 <= system_choice <= 5:
                    self.system_type = system_choice
                    break
                else:
                    print("Пожалуйста, введите число от 1 до 5")
            except ValueError:
                print("Ошибка: введите число от 1 до 5")

        # Выбор типа возмущения
        print("\nДоступные типы начальных возмущений:")
        print("1. Точечный шум в определенном проценте клеток по маске")
        print("2. Равномерный шум")
        print("3.Явное возмущение в центре")
        print("4. Гауссово пятно в центре")

        while True:
            try:
                perturbation_choice = int(input("Выберите тип возмущения (1-4): "))
                if 1 <= perturbation_choice <= 4:
                    self.perturbation_type = perturbation_choice
                    break
                else:
                    print("Пожалуйста, введите число от 1 до 4")
            except ValueError:
                print("Ошибка: введите число от 1 до 4")

    #----------------ПОСТАВЬТЕ СВОИ ПАРАМЕТРЫ РАСЧЕТНОЙ ОБЛАСТИ И ВОЗМУЩЕНИЙ, ЕСЛИ НУЖНО----------------#
        # Параметры общие для всех систем
        self.L =0.5
        self.N = 100  # Размер сетки NxN
        self.dt = 0.5e-7  # шаг по времени по умолчанию, если не выбран другой в диалоге
        self.total_steps = 110000 # Сколько всего шагов будет сделано

        # Параметры шума (общие для всех систем)
        self.U_fl = 0.02
        self.V_fl = 0.02
        self.Y_fl = 0.02

        self.noise_fractionU = 0.5 #(процент возмущенных клеток)
        self.noise_fractionV = 0.5
        self.noise_fractionY = 0.5


        #------------ПОСТАВЬТЕ СВОИ ПАРАМЕТРЫ В ЗАВИСИМОСТИ ОТ ВЫБРАННОЙ СИСТЕМЫ------------#

        # Установка параметров
        if self.system_type == 1:
            self.k = 0.55
            self.c = 0.01
            self.e = 0.02
            self.S = 0
            self.Du = 0.01
            self.Dv = 2
            self.U0 = 1.1
            self.V0 = 61.25
            self.gamma = 30000
            print("\nВыбрана система Мейнхардта")

        elif self.system_type == 2:  # Шнакенберга
            # #Параметры для демонстрации паттернов. Гауссово пятно
            self.L =1
            self.N = 100  # Размер сетки NxN
            self.dt = 1e-7  # шаг по времени
            self.total_steps = 110000
            self.a = 0.1
            self.b = 0.9
            self.Du = 0.01
            self.Dv = 1
            self.U0 = 1.0
            self.V0 = 1.0
            self.gamma = 60000
            print("\nВыбрана система Шнакенберга")

        elif self.system_type == 3:  # Нелинейная система из статьи
            self.a = 0.1
            self.b = 0.9
            self.Du = 0.01
            self.Dv = 1
            self.U0 = 1.0
            self.V0 = 1.0
            self.gamma = 60000
            print("\nВыбрана cистема с кубическим членом в активаторе")

        elif self.system_type == 4:  # Система для с третьим полем Y
            self.L = 120
            self.N = 120  # Размер сетки NxN
            self.Ka = 0.5
            self.Ks = 0.3
            self.Ky = 22.0
            self.Rho_a = 0.05
            self.Rho_s = 0.0035
            self.Rho_y = 0.03
            self.sigma_s = 0.0075
            self.sigma_y = 0.00007
            self.mu_s = 0.003
            self.mu_y = 0.003
            self.Du = 0.01
            self.Dv = 0.1
            self.Dy = 0.1
            self.U0 = 0
            self.V0 = 3
            self.Y0 = 0
            self.pert = 0.25
            print("\nВыбрана cистема с третьим полем Y")

        elif self.system_type == 5:  # Система "круговая область"
            self.L = 50
            self.N = 100
            self.a =0  # Пример значения, можно изменить
            self.b = 0  # Пример значения, можно изменить
            self.Du = 1  # Коэффициент диффузии активатора
            self.Dv = 8.0   # Коэффициент диффузии ингибитора (D = 8 из статьи)
            self.U0 = 1.0   # Начальное значение U
            self.V0 = 1.0   # Начальное значение V
            self.gamma = 1
            print("\nВыбрана система 'круговая  область'")


        # Инициализация массивов с выбранным типом возмущения
        self._initialize_perturbation()

        # Остальные параметры инициализации

        self.dx = self.L / self.N

        self.frame_files = []
        self.partial_gif_counter = 0
        self.step_count = 0
        self.prev_U = None
        self.prev_V = None
        self.prev_Y = None
        self.stable_steps = 0
        self.required_stable_steps = 1000
        self.stability_threshold = 1e-6


    # Инициализация массивов с выбранным типом возмущения
    def _initialize_perturbation(self):
        np.random.seed(3425)  # Фиксируем seed для воспроизводимости
        # Начальная инициализация
        self.U = self.U0 * np.ones((self.N, self.N))
        self.V = self.V0 * np.ones((self.N, self.N))

        if self.system_type == 4:  # Только для системы 4 инициализируем Y
            self.Y = self.Y0 * np.ones((self.N, self.N))

        # Вариант 1: Точечный шум в определенном проценте клеток по маске
        if self.perturbation_type == 1:
            if self.system_type == 4:
                N2 = self.N // 2
                r = int(self.N / 20.0)
                self.Y = self.Y0 * np.ones((self.N, self.N))

                # Создаем случайные точечные возмущения
                for k in range(0, N2):
                    i = np.random.randint(0, self.N - 1)
                    j = np.random.randint(0, self.N - 1)
                    self.U[i, j] = 5.0  # Повышенное значение, аналогично model.up в первом коде
                    self.Y[i, j] = 1.0   # Аналогично первому коду

                  # Добавляем небольшой шум ко всем переменным
                self.U += self.U_fl * self.U0 * (2 * np.random.random((self.N, self.N)) - 1)
                self.V += self.V_fl * self.V0 * (2 * np.random.random((self.N, self.N)) - 1)
                self.Y += self.pert * self.Y0 * (2 * np.random.random((self.N, self.N)) - 1)
            else:
                U_noise_mask = np.random.random(size=(self.N, self.N)) < self.noise_fractionU
                V_noise_mask = np.random.random(size=(self.N, self.N)) < self.noise_fractionV
                self.U[U_noise_mask] += (2 * np.random.random(size=(self.N, self.N))[U_noise_mask] - 1) * self.U_fl * self.U0
                self.V[V_noise_mask] += (2 * np.random.random(size=(self.N, self.N))[V_noise_mask] - 1) * self.V_fl * self.V0

        # Вариант 2: равномерный шум
        elif self.perturbation_type == 2:
            self.U = self.U0 + self.U_fl * self.U0 * (2 * np.random.random((self.N, self.N)) - 1)
            self.V = self.V0 + self.V_fl * self.V0 * (2 * np.random.random((self.N, self.N)) - 1)
            if self.system_type == 4:
                self.Y = self.Y0 + 0.1 * self.Y0 * (2 * np.random.random((self.N, self.N)) - 1)

        # Вариант 3: Сильное возмущение в центре
        elif self.perturbation_type == 3:
            N2 = self.N // 2
            r = int(0.05*self.N)  # Размер пятна ~5% от сетки
            self.U[N2-r:N2+r, N2-r:N2+r] = 2  # Сильное возмущение U
            self.V[N2-r:N2+r, N2-r:N2+r] = 2.5  # Возмущение V

            # Добавляем шум ко всей области
            self.U += self.U_fl * self.U0 * (2 * np.random.random((self.N, self.N)) - 1)
            self.V += self.V_fl * self.V0 * (2 * np.random.random((self.N, self.N)) - 1)

            if self.system_type == 4:
                self.Y[N2-r:N2+r, N2-r:N2+r] = 1.0  # Возмущение Y
                self.Y += self.pert * self.Y0 * (2 * np.random.random((self.N, self.N)) - 1)

        # Вариант 4: Гауссово пятно в центре
        elif self.perturbation_type == 4:
            def gaussian_perturbation(N, amplitude, sigma=10):
                x = np.linspace(0, N, N)
                y = np.linspace(0, N, N)
                X, Y = np.meshgrid(x, y)
                return amplitude * np.exp(-((X - N//2)**2 + (Y - N//2)**2) / (2 * sigma**2))

            self.U = self.U0 + gaussian_perturbation(self.N, 0.1 * self.U0)
            self.V = self.V0 + gaussian_perturbation(self.N, 0.1 * self.V0)
            if self.system_type == 4:
                self.Y = self.Y0 + gaussian_perturbation(self.N, 0.1 * self.Y0)

        # Применяем non_negative только к инициализированным переменным
        if self.system_type == 4:
            self.U, self.V, self.Y = self._non_negative(self.U, self.V, self.Y)
        else:
            self.U, self.V = self._non_negative(self.U, self.V)

        # Для анимации
        self.frame_files = []  # Будем хранить пути к временным файлам
        self.partial_gif_counter = 0
        self.step_count = 0

        # Для проверки стабилизации
        self.prev_U = None
        self.prev_V = None
        self.prev_Y = None if self.system_type == 4 else None
        self.stable_steps = 0
        self.required_stable_steps = 1000  # Количество шагов подряд для подтверждения стабилизации
        self.stability_threshold = 1e-6  # Порог изменения для стабилизации


    # Проверка и исправление отрицательных значений в массивах.
    # Возвращает исправленные массивы в том же порядке
    def _non_negative(self, *arrays):
        result = []
        for arr in arrays:
            #arr[arr <= 0] = arr[arr <= 0]  # Оставляем отрицательные и нулевые концентрации
            arr[arr <= 0] = 1e-10  # Заменяем отрицательные значения на очень маленькое положительное
            result.append(arr)
        return tuple(result) if len(result) > 1 else result[0]


    # Проверка условий Тьюринга для выбранной системы
    #------------ЕСЛИ БЫЛИ ИЗМЕНЕНЫ РЕАКЦИОННЫЕ ЧАСТИ УРАВНЕНИЙ, ВЫЧИСЛИТЕ ИХ ЧАСТНЫЕ ПРОИЗВОДНЫЕ-----#
    #------------И ВПИШИТЕ НИЖЕ В СООТВЕТСТВИИ С ВЫБРАННОЙ СИСТЕМОЙ ----------------------------------#
    def check_turing_conditions(self):
          print("\nПроверка условий Тьюринга для выбранной системы:")

          if self.system_type == 1:  # Мейнхардта
              # Стационарные состояния (u0, v0) находятся из f(u0,v0)=0, g(u0,v0)=0
              # Для нашей системы это сложно аналитически, поэтому используем начальные значения
              u0, v0 = self.U0, self.V0

              # Вычисляем частные производные в стационарной точке
              denominator = (1 + self.k * u0**2) * v0
              f_u = self.gamma * ((2*u0*(1 + self.k*u0**2)-2*self.k*u0**3)/(v0*(1 + self.k*u0**2)**2) - self.c)
              f_v = -self.gamma * (u0**2/((1 + self.k*u0**2)*v0**2))
              g_u = self.gamma * 2*u0
              g_v = -self.gamma * self.e

              # Условия Тьюринга
              trace_J = f_u + g_v
              det_J = f_u * g_v - f_v * g_u

              print(f"Для системы 1 (Мейнхардта):")
              print(f"f_u = {f_u:.4f}, f_v = {f_v:.4f}")
              print(f"g_u = {g_u:.4f}, g_v = {g_v:.4f}")
              print(f"След матрицы Якоби (tr(J)) = {trace_J:.4f} (должен быть < 0)")
              print(f"Определитель матрицы Якоби (det(J)) = {det_J:.4f} (должен быть > 0)")

              if trace_J < 0 and det_J > 0:
                  print("Условия устойчивости однородного состояния ВЫПОЛНЕНЫ")

                  # Проверка условия диффузионной неустойчивости
                  D_ratio = self.Dv / self.Du
                  turing_condition = (f_u * self.Dv + g_v * self.Du) ** 2 - 4 * self.Du * self.Dv * det_J
                  print(f"Условие диффузионной неустойчивости: {turing_condition:.4f} (должно быть > 0)")

                  if turing_condition > 0:
                      print("Условия Тьюринга ВЫПОЛНЕНЫ - возможны паттерны!")
                      return True
                  else:
                      print("Условия Тьюринга НЕ ВЫПОЛНЕНЫ - паттерны маловероятны")
                      return False
              else:
                  print("Однородное состояние НЕУСТОЙЧИВО - условия Тьюринга неприменимы")
                  return False

          elif self.system_type == 2:  # Шнакенберга
              # Стационарные состояния
              u0 = self.a + self.b
              v0 = self.b / (self.a + self.b)**2

              # Частные производные
              f_u = self.gamma * (-1 + 2*u0*v0)
              f_v = self.gamma * u0**2
              g_u = self.gamma * (-2*u0*v0)
              g_v = self.gamma * (-u0**2)

              # Условия Тьюринга
              trace_J = f_u + g_v
              det_J = f_u * g_v - f_v * g_u

              print(f"Для системы 2 (Шнакенберга):")
              print(f"f_u = {f_u:.4f}, f_v = {f_v:.4f}")
              print(f"g_u = {g_u:.4f}, g_v = {g_v:.4f}")
              print(f"След матрицы Якоби (tr(J)) = {trace_J:.4f} (должен быть < 0)")
              print(f"Определитель матрицы Якоби (det(J)) = {det_J:.4f} (должен быть > 0)")

              if trace_J < 0 and det_J > 0:
                  print("Условия устойчивости однородного состояния ВЫПОЛНЕНЫ")

                  # Проверка условия диффузионной неустойчивости
                  D_ratio = self.Dv / self.Du
                  turing_condition = (f_u * self.Dv + g_v * self.Du) ** 2 - 4 * self.Du * self.Dv * det_J
                  print(f"Условие диффузионной неустойчивости: {turing_condition:.4f} (должно быть > 0)")
                  print(f"f_u/Du = {f_u/self.Du:.4f}, g_v/Dv = {g_v/self.Dv:.4f}")
                  print(f"Необходимо: f_u > 0, g_v < 0, Dv > Du")

                  if turing_condition > 0 and f_u > 0 and g_v < 0 and self.Dv > self.Du:
                      print("Условия Тьюринга ВЫПОЛНЕНЫ - возможны паттерны!")
                      return True
                  else:
                      print("Условия Тьюринга НЕ ВЫПОЛНЕНЫ - паттерны маловероятны")
                      return False
              else:
                  print("Однородное состояние НЕУСТОЙЧИВО - условия Тьюринга неприменимы")
                  return False

          elif self.system_type == 3:  # Нелинейная система с кубическим членом
              # Стационарные состояния
              u0 = self.a + self.b
              v0 = self.b / (self.a + self.b)**2

              # Частные производные
              f_u = self.gamma * (-1 + 2*u0*v0 - 3*u0**2)
              f_v = self.gamma * u0**2
              g_u = self.gamma * (-2*u0*v0)
              g_v = self.gamma * (-u0**2)

              # Условия Тьюринга
              trace_J = f_u + g_v
              det_J = f_u * g_v - f_v * g_u

              print(f"Для системы 3 (Нелинейная система):")
              print(f"f_u = {f_u:.4f}, f_v = {f_v:.4f}")
              print(f"g_u = {g_u:.4f}, g_v = {g_v:.4f}")
              print(f"След матрицы Якоби (tr(J)) = {trace_J:.4f} (должен быть < 0)")
              print(f"Определитель матрицы Якоби (det(J)) = {det_J:.4f} (должен быть > 0)")

              if trace_J < 0 and det_J > 0:
                  print("Условия устойчивости однородного состояния ВЫПОЛНЕНЫ")

                  # Проверка условия диффузионной неустойчивости
                  D_ratio = self.Dv / self.Du
                  turing_condition = (f_u * self.Dv + g_v * self.Du) ** 2 - 4 * self.Du * self.Dv * det_J
                  print(f"Условие диффузионной неустойчивости: {turing_condition:.4f} (должно быть > 0)")
                  print(f"f_u/Du = {f_u/self.Du:.4f}, g_v/Dv = {g_v/self.Dv:.4f}")
                  print(f"Необходимо: f_u > 0, g_v < 0, Dv > Du")

                  if turing_condition > 0 and f_u > 0 and g_v < 0 and self.Dv > self.Du:
                      print("Условия Тьюринга ВЫПОЛНЕНЫ - возможны паттерны!")
                      return True
                  else:
                      print("Условия Тьюринга НЕ ВЫПОЛНЕНЫ - паттерны маловероятны")
                      return False
              else:
                  print("Однородное состояние НЕУСТОЙЧИВО - условия Тьюринга неприменимы")
                  return False

          elif self.system_type == 4:  # Система с третьим полем Y
              print("Для системы 4 (системы с третьим полем Y) проверка условий Тьюринга сложна из-за нелинейностей")
              print("и третьей переменной. Проверка не реализована в этой версии.")
              return True  # Возвращаем True, чтобы продолжить моделирование

          elif self.system_type == 5:  # Система "круговая область"
              # Стационарные состояния
              u0 = 1.0
              v0 = 1.0

              # Частные производные
              f_u = self.gamma * (2*u0 - v0)
              f_v = self.gamma * (-u0)
              g_u = self.gamma * (- (2 - self.a**2)*v0)
              g_v = self.gamma * (-(1 - self.a**2 - self.b**2) - (2 - self.a**2)*u0)

              # Условия Тьюринга
              trace_J = f_u + g_v
              det_J = f_u * g_v - f_v * g_u

              print(f"Для системы 5 (круговой области):")
              print(f"f_u = {f_u:.4f}, f_v = {f_v:.4f}")
              print(f"g_u = {g_u:.4f}, g_v = {g_v:.4f}")
              print(f"След матрицы Якоби (tr(J)) = {trace_J:.4f} (должен быть < 0)")
              print(f"Определитель матрицы Якоби (det(J)) = {det_J:.4f} (должен быть > 0)")

              if trace_J < 0 and det_J > 0:
                  print("Условия устойчивости однородного состояния ВЫПОЛНЕНЫ")

                  # Проверка условия диффузионной неустойчивости
                  D_ratio = self.Dv / self.Du
                  turing_condition = (f_u * self.Dv + g_v * self.Du) ** 2 - 4 * self.Du * self.Dv * det_J
                  print(f"Условие диффузионной неустойчивости: {turing_condition:.4f} (должно быть > 0)")
                  print(f"f_u/Du = {f_u/self.Du:.4f}, g_v/Dv = {g_v/self.Dv:.4f}")
                  print(f"Необходимо: f_u > 0, g_v < 0, Dv > Du")

                  if turing_condition > 0 and f_u > 0 and g_v < 0 and self.Dv > self.Du:
                      print("Условия Тьюринга ВЫПОЛНЕНЫ - возможны паттерны!")
                      return True
                  else:
                      print("Условия Тьюринга НЕ ВЫПОЛНЕНЫ - паттерны маловероятны")
                      return False
              else:
                  print("Однородное состояние НЕУСТОЙЧИВО - условия Тьюринга неприменимы")
                  return False

    # Вычисление лапласиана с граничными условиями Неймана
    def laplacian(self, Z):
        Ztop = Z[0:-2, 1:-1]
        Zleft = Z[1:-1, 0:-2]
        Zbottom = Z[2:, 1:-1]
        Zright = Z[1:-1, 2:]
        Zcenter = Z[1:-1, 1:-1]
        numerator = Ztop + Zleft + Zbottom + Zright - 4*Zcenter
        return numerator / (self.dx**2 + 1e-10)

    # Вычисление нелинейных реакционных членов в зависимости от выбранной системы
    def reaction_terms(self, U, V, Y=None):
        # Проверка входных данных
        U_safe, V_safe = self._non_negative(U, V)
        if Y is not None:
            Y_safe = self._non_negative(Y)

        if np.any(np.isnan(U_safe)) or np.any(np.isnan(V_safe)) or (Y is not None and np.any(np.isnan(Y_safe))):
            raise ValueError("Обнаружены NaN значения в концентрациях")

        U_center = U_safe[1:-1,1:-1]
        V_center = V_safe[1:-1,1:-1]

        #------------ЕСЛИ БЫЛИ ИЗМЕНЕНЫ РЕАКЦИОННЫЕ ЧАСТИ УРАВНЕНИЙ, ВПИШИТЕ ИХ НИЖЕ В СООТВЕТСТВИИ С ВЫБРАННОЙ СИСТЕМОЙ ------------#
        try:
            if self.system_type == 1: #Мейнхардта
                U_sq = U_center**2
                denominator = (1 + self.k*U_sq)*V_center
                denominator = np.clip(denominator, 1e-10, None)  # Защита от деления на ноль
                f = self.gamma * (U_sq / denominator - self.c*U_center)
                g = self.gamma * (U_sq - self.e*V_center + self.S)

            elif self.system_type == 2:  # Шнакенберга
                f = self.gamma * (self.a - U_center + U_center**2 * V_center)
                g = self.gamma * (self.b - U_center**2 * V_center)

            elif self.system_type == 3:  # Нелинейная система с кубическим членом
                f = self.gamma * (self.a - U_center + U_center**2 * V_center - U_center**3)
                g = self.gamma * (self.b - U_center**2 * V_center)

            elif self.system_type == 4:  # Система с третьим полем Y
                if Y is None:
                    raise ValueError("Для системы 4 параметр Y обязателен")
                Y_safe = np.clip(Y, 1e-10, None)
                Y_center = Y_safe[1:-1,1:-1]

                denominator1 = np.maximum(1e-10, 1 + self.Ka * U_center**2)
                denominator2 = np.maximum(1e-10, 1 + self.Ks * Y_center)
                denominator3 = np.maximum(1e-10, 1 + self.Ky * Y_center**2)

                f = self.Rho_a * ((U_center**2 * V_center)/denominator1 - U_center)
                g = (self.sigma_s/denominator2 - self.Rho_s*U_center**2*V_center/denominator1 -
                    self.mu_s*V_center)
                y_term = (self.Rho_y*Y_center**2/denominator3 - self.mu_y*Y_center +
                        self.sigma_y*U_center)
                return f, g, y_term

            elif self.system_type == 5:  # Система "круговая область"
                f = self.gamma * (U_center**2 - U_center*V_center)
                g = self.gamma * ( 1 + self.b**2 -(1 - self.a**2 - self.b**2)*V_center - (2 - self.a**2)*U_center*V_center)

            # Проверка результатов вычислений
            if np.any(np.isnan(f)) or np.any(np.isnan(g)) or (self.system_type == 4 and np.any(np.isnan(y_term))):
                raise ValueError("Реакционные члены содержат NaN значения")

            if np.any(np.abs(f) > 1e10) or np.any(np.abs(g) > 1e10) or (self.system_type == 4 and np.any(np.abs(y_term) > 1e10)):
                raise ValueError("Реакционные члены слишком большие по величине")

        except FloatingPointError as e:
            raise ValueError(f"Ошибка при вычислении реакционных членов: {str(e)}")

        if self.system_type == 4:
            return f, g, y_term

        return f, g


    # Проверка стабилизации системы
    def check_stabilization(self):
        # Для системы 4  проверяем все три переменные
        if self.system_type == 4 :
            if self.prev_Y is None:
                self.prev_U = self.U.copy()
                self.prev_V = self.V.copy()
                self.prev_Y = self.Y.copy()
                return False

            delta_U = np.max(np.abs(self.U - self.prev_U))
            delta_V = np.max(np.abs(self.V - self.prev_V))
            delta_Y = np.max(np.abs(self.Y - self.prev_Y))

            self.prev_U = self.U.copy()
            self.prev_V = self.V.copy()
            self.prev_Y = self.Y.copy()

            if (delta_U < self.stability_threshold and
                delta_V < self.stability_threshold and
                delta_Y < self.stability_threshold):
                self.stable_steps += 1
                return self.stable_steps >= self.required_stable_steps

        # Для систем 1-3,5 проверяем только U и V
        else:
            if self.prev_U is None or self.prev_V is None:
                self.prev_U = self.U.copy()
                self.prev_V = self.V.copy()
                return False

            delta_U = np.max(np.abs(self.U - self.prev_U))
            delta_V = np.max(np.abs(self.V - self.prev_V))

            self.prev_U = self.U.copy()
            self.prev_V = self.V.copy()

            if (delta_U < self.stability_threshold and
                delta_V < self.stability_threshold):
                self.stable_steps += 1
                return self.stable_steps >= self.required_stable_steps

        self.stable_steps = 0
        return False


    # Метод Рунге-Кутты 2-го порядка (модифицированный Эйлер) с защитой от отрицательных концентраций
    def rk2_step(self):

        # Система 4 - 3 переменные (U, V, Y)
        if self.system_type == 4:
            # Шаг 1: Предиктор (k1) — полный шаг dt
            f1, g1, y1 = self.reaction_terms(self.U, self.V, self.Y)
            lap_U1 = self.laplacian(self.U)
            lap_V1 = self.laplacian(self.V)
            lap_Y1 = self.laplacian(self.Y)

            # Временные U1, V1, Y1 (k1)
            U1 = self.U.copy()
            V1 = self.V.copy()
            Y1 = self.Y.copy()
            U1[1:-1, 1:-1] += self.dt * (self.Du * lap_U1 + f1)
            V1[1:-1, 1:-1] += self.dt * (self.Dv * lap_V1 + g1)
            Y1[1:-1, 1:-1] += self.dt * (self.Dy * lap_Y1 + y1)

            # Защита от отрицательных концентраций
            U1, V1, Y1 = self._non_negative(U1, V1, Y1)

            # Граничные условия для U1, V1, Y1
            for Z in [U1, V1, Y1]:
                Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
                Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

            # Шаг 2: Корректор (k2) — усреднение k1 и k2
            f2, g2, y2 = self.reaction_terms(U1, V1, Y1)
            lap_U2 = self.laplacian(U1)
            lap_V2 = self.laplacian(V1)
            lap_Y2 = self.laplacian(Y1)

            # Итоговое обновление: U += 0.5*dt*(k1 + k2)
            self.U[1:-1, 1:-1] += 0.5 * self.dt * (self.Du * (lap_U1 + lap_U2) + (f1 + f2))
            self.V[1:-1, 1:-1] += 0.5 * self.dt * (self.Dv * (lap_V1 + lap_V2) + (g1 + g2))
            self.Y[1:-1, 1:-1] += 0.5 * self.dt * (self.Dy * (lap_Y1 + lap_Y2) + (y1 + y2))

            U1, V1, Y1 = self._non_negative(U1, V1, Y1)
            # Граничные условия для новых U, V, Y
            for Z in [self.U, self.V, self.Y]:
                Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
                Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]


        # Системы 1-3,5 (2 переменные U, V)
        else:
            # Шаг 1: Предиктор (k1) — полный шаг dt
            f1, g1 = self.reaction_terms(self.U, self.V)
            lap_U1 = self.laplacian(self.U)
            lap_V1 = self.laplacian(self.V)

            # Временные U1, V1 (k1)
            U1 = self.U.copy()
            V1 = self.V.copy()
            U1[1:-1, 1:-1] += self.dt * (self.Du * lap_U1 + f1)
            V1[1:-1, 1:-1] += self.dt * (self.Dv * lap_V1 + g1)

            self.U, self.V = self._non_negative(self.U, self.V)

            # Граничные условия для U1, V1
            for Z in [U1, V1]:
                Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
                Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

            # Шаг 2: Корректор (k2) — усреднение k1 и k2
            f2, g2 = self.reaction_terms(U1, V1)
            lap_U2 = self.laplacian(U1)
            lap_V2 = self.laplacian(V1)

            # Итоговое обновление: U += 0.5*dt*(k1 + k2)
            self.U[1:-1, 1:-1] += 0.5 * self.dt * (self.Du * (lap_U1 + lap_U2) + (f1 + f2))
            self.V[1:-1, 1:-1] += 0.5 * self.dt * (self.Dv * (lap_V1 + lap_V2) + (g1 + g2))

            self.U, self.V = self._non_negative(self.U, self.V)

            # Граничные условия для новых U, V
            for Z in [self.U, self.V]:
                Z[0, :] = Z[1, :]; Z[-1, :] = Z[-2, :]
                Z[:, 0] = Z[:, 1]; Z[:, -1] = Z[:, -2]

        self.step_count += 1


    # Улучшенный метод контрастирования с адаптивной бинаризацией
    def _enhance_contrast(self, data):
        filtered = ndimage.median_filter(data, size=3)
        p5, p95 = np.percentile(filtered, [5, 95])
        normalized = (filtered - p5) / (p95 - p5 + 1e-10)
        normalized = np.clip(normalized, 0, 1)

        # Адаптивная бинаризация
        block_size = 41
        local_thresh = threshold_local(normalized, block_size=block_size)
        binary = normalized > local_thresh

        # Морфологическая обработка
        binary = morphology.binary_opening(binary, footprint=np.ones((2,2)))
        binary = morphology.remove_small_objects(binary, min_size=8)
        binary = morphology.remove_small_holes(binary, area_threshold=8)

        return binary


    # Визуализация текущего состояния системы с цветными и бинарными изображениями
    def show_state(self, step):
        plt.figure(figsize=(18, 16), dpi=100)

        # Применяем бинаризацию
        U_binary = self._enhance_contrast(self.U)
        V_binary = self._enhance_contrast(self.V)
        UV_binary = self._enhance_contrast(self.U - self.V)

        # Первый ряд: активатор U
        plt.subplot(4, 3, 1)
        plt.imshow(self.U, cmap='viridis', interpolation='bilinear')
        plt.title(f'Активатор U ( шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 2)
        plt.imshow(self.U, cmap='binary', interpolation='bilinear')
        plt.title(f'Активатор U ( шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 3)
        plt.imshow(U_binary, cmap='binary', interpolation='none')
        plt.title(f'Активатор U ( шаг {step})')

        # Второй ряд: ингибитор V
        plt.subplot(4, 3, 4)
        plt.imshow(self.V, cmap='viridis', interpolation='bilinear')
        plt.title(f'Ингибитор V ( шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 5)
        plt.imshow(self.V, cmap='binary', interpolation='bilinear')
        plt.title(f'Ингибитор V (шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 6)
        plt.imshow(V_binary, cmap='binary', interpolation='none')
        plt.title(f'Ингибитор V ( шаг {step})')

        # Третий ряд: система U-V
        plt.subplot(4, 3, 7)
        plt.imshow(self.U - self.V, cmap='viridis', interpolation='bilinear')
        plt.title(f'Система U-V ( шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 8)
        plt.imshow(self.U - self.V, cmap='binary', interpolation='bilinear')
        plt.title(f'Система U-V ( шаг {step})')
        plt.colorbar()

        plt.subplot(4, 3, 9)
        plt.imshow(UV_binary, cmap='binary', interpolation='none')
        plt.title(f'Система U-V ( шаг {step})')

        # Четвертый ряд: графики концентраций
        plt.subplot(4,3, 10)
        center = self.N // 2
        epsilon = 1e-10  # очень маленькое число
        U_norm = self.U[center, :] / (np.max(self.U) + epsilon)
        V_norm = self.V[center, :] / (np.max(self.V) + epsilon)
        plt.plot(U_norm, label='Норм. активатор (U)')
        plt.plot(V_norm, label='Норм. ингибитор (V)')
        plt.legend()
        plt.title('Профили концентраций по центру')
        plt.xlabel('Позиция')
        plt.ylabel('Нормированная концентрация')
        plt.grid(True)
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        plt.show()


    # Сохранение кадра для GIF с идентичной визуализацией, как в show_state
    def _save_frame_to_temp_file(self, step, output_dir):
        Path(output_dir).mkdir(exist_ok=True)

        # Создаем фигуру с теми же параметрами, что и в show_state
        plt.figure(figsize=(18, 16), dpi=100)

        # --- Первый ряд: активатор U ---
        plt.subplot(4, 3, 1)
        img_u = plt.imshow(self.U, cmap='viridis', interpolation='bilinear')
        plt.title(f'Активатор U (цвет, шаг {step})')
        # plt.colorbar(img_u)

        plt.subplot(4, 3, 2)
        img_u_bin = plt.imshow(self.U, cmap='binary', interpolation='bilinear')
        plt.title(f'Активатор U (бинарный, шаг {step})')
        # plt.colorbar(img_u_bin)

        plt.subplot(4, 3, 3)
        U_binary = self._enhance_contrast(self.U)
        plt.imshow(U_binary, cmap='binary', interpolation='none')
        plt.title(f'Активатор U (обработанный, шаг {step})')

        # --- Второй ряд: ингибитор V ---
        plt.subplot(4, 3, 4)
        img_v = plt.imshow(self.V, cmap='viridis', interpolation='bilinear')
        plt.title(f'Ингибитор V (цвет, шаг {step})')
        # plt.colorbar(img_v)

        plt.subplot(4, 3, 5)
        img_v_bin = plt.imshow(self.V, cmap='binary', interpolation='bilinear')
        plt.title(f'Ингибитор V (бинарный, шаг {step})')
        # plt.colorbar(img_v_bin)

        plt.subplot(4, 3, 6)
        V_binary = self._enhance_contrast(self.V)
        plt.imshow(V_binary, cmap='binary', interpolation='none')
        plt.title(f'Ингибитор V (обработанный, шаг {step})')

        # --- Третий ряд: система U-V ---
        plt.subplot(4, 3, 7)
        uv_diff = self.U - self.V
        img_uv = plt.imshow(uv_diff, cmap='viridis', interpolation='bilinear')
        plt.title(f'Система U-V (цвет, шаг {step})')
        # plt.colorbar(img_uv)

        plt.subplot(4, 3, 8)
        img_uv_bin = plt.imshow(uv_diff, cmap='binary', interpolation='bilinear')
        plt.title(f'Система U-V (бинарный, шаг {step})')
        # plt.colorbar(img_uv_bin)

        plt.subplot(4, 3, 9)
        UV_binary = self._enhance_contrast(uv_diff)
        plt.imshow(UV_binary, cmap='binary', interpolation='none')
        plt.title(f'Система U-V (обработанный, шаг {step})')

        # --- Четвертый ряд: графики концентраций ---
        plt.subplot(4, 3, 10)
        center = self.N // 2
        epsilon = 1e-10
        U_norm = self.U[center, :] / (np.max(self.U) + epsilon)
        V_norm = self.V[center, :] / (np.max(self.V) + epsilon)
        plt.plot(U_norm, label='Норм. активатор (U)')
        plt.plot(V_norm, label='Норм. ингибитор (V)')
        plt.legend()
        plt.title('Профили концентраций по центру')
        # plt.xlabel('Позиция')
        # plt.ylabel('Нормированная концентрация')
        # plt.grid(True)
        plt.axis('off')
        # Сохранение с теми же параметрами
        plt.tight_layout(pad=3.0)
        frame_path = os.path.join(output_dir, f"frame_{step:06d}.png")
        plt.savefig(frame_path, bbox_inches='tight', dpi=100)  # Теперь dpi=100 как в show_state
        plt.close()

        return frame_path

    # Создает частичный GIF из временных файлов и очищает их
    def save_partial_gif(self, output_dir):
        if not self.frame_files:
            return

        Path(output_dir).mkdir(exist_ok=True)
        partial_path = os.path.join(output_dir, f'partial_{self.partial_gif_counter}.gif')

        # Читаем все кадры в память и создаем GIF
        with imageio.get_writer(partial_path, mode='I', duration=0.1) as writer:
            for frame_file in self.frame_files:
                image = imageio.imread(frame_file)
                writer.append_data(image)
                os.remove(frame_file)  # Удаляем временный файл

        print(f"Создан частичный GIF: {partial_path}")
        self.partial_gif_counter += 1
        self.frame_files = []

    # Собирает финальный GIF из частичных GIF
    def create_final_animation(self, output_dir):
        if self.partial_gif_counter == 0 and not self.frame_files:
            print("Нет данных для создания анимации.")
            return

        final_path = os.path.join(output_dir, 'final_animation.gif')

        # Создаем финальный GIF
        with imageio.get_writer(final_path, mode='I', duration=0.1) as writer:
            # Добавляем все частичные GIF
            for i in range(self.partial_gif_counter):
                part_path = os.path.join(output_dir, f'partial_{i}.gif')
                try:
                    with imageio.get_reader(part_path) as reader:
                        for frame in reader:
                            writer.append_data(frame)
                    os.remove(part_path)
                except FileNotFoundError:
                    print(f"Файл {part_path} не найден, пропускаем.")

            # Добавляем оставшиеся кадры, если они есть
            for frame_file in self.frame_files:
                try:
                    image = imageio.imread(frame_file)
                    writer.append_data(image)
                    os.remove(frame_file)
                except FileNotFoundError:
                    print(f"Файл {frame_file} не найден, пропускаем.")

        self.frame_files = []
        print(f"Финальная анимация сохранена: {final_path}")

    # Сохранение результатов с максимальной четкостью
    def save_final_patterns(self, output_dir):
        Path(output_dir).mkdir(exist_ok=True)

        # Применяем улучшенную бинаризацию
        U_binary = self._enhance_contrast(self.U)
        V_binary = self._enhance_contrast(self.V)
        UV_binary = self._enhance_contrast(self.U - self.V)

        # Сохраняем все варианты изображений
        plt.imsave(os.path.join(output_dir, 'final_U_binary.png'),
                 U_binary, cmap='binary', dpi=300)
        plt.imsave(os.path.join(output_dir, 'final_V_binary.png'),
                 V_binary, cmap='binary', dpi=300)
        plt.imsave(os.path.join(output_dir, 'final_UV_binary.png'),
                 UV_binary, cmap='binary', dpi=300)

        # Оригиналы в градациях серого
        plt.imsave(os.path.join(output_dir, 'final_U_original.png'),
                 self.U, cmap='viridis', dpi=300)
        plt.imsave(os.path.join(output_dir, 'final_V_original.png'),
                 self.V, cmap='viridis', dpi=300)

        # Сохраняем данные в numpy формате
        np.savez(os.path.join(output_dir, 'final_data.npz'),
                U=self.U, V=self.V, U_binary=U_binary)



    # Основной цикл моделирования
    def run_simulation(self, output_dir="results",
                     display_interval=1000,
                     gif_save_interval=50000,
                     full_display_interval=10000):
        # Создаем папку для результатов с номером системы
        output_dir = os.path.join("results", f"system_{self.system_type}")
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        print(f"\nРезультаты будут сохранены в: {output_dir}")

        # Проверка условий Тьюринга перед началом моделирования
        turing_ok = self.check_turing_conditions()
        if not turing_ok:
            choice = input("Условия Тьюринга не выполнены. Продолжить моделирование? (y/n): ").strip().lower()
            if choice != 'y':
                print("Моделирование прервано пользователем")
                return

        print("Начальное состояние системы:")
        self.show_state(0)
        if self.with_animation:
            self.frame_files.append(self._save_frame_to_temp_file(0, output_dir))

        # Проверка числа Куранта перед началом моделирования
        print("\nПроверка числа Куранта (устойчивость):")
        dt_max = (self.dx**2) / (4 * max(self.Du, self.Dv))
        courant = self.dt / dt_max
        print(f"Текущий dt = {self.dt:.2e}, максимально допустимый dt = {dt_max:.2e}")
        print(f"Число Куранта = {courant:.4f} (должно быть < 1): {'выполнено' if courant < 1 else 'НЕ ВЫПОЛНЕНО!'}")

        # Тест на сходимость (проверка изменения решения при уменьшении шага)
        print("\nПроверка сходимости:")
        test_steps = 10000
        original_dt = self.dt

        # Сохраняем начальные условия
        U_orig = self.U.copy()
        V_orig = self.V.copy()

        # Запускаем с текущим шагом
        for _ in range(test_steps):
            self.rk2_step()
        result1 = self.U.copy()

        # Возвращаем начальные условия и уменьшаем шаг вдвое
        self.U = U_orig.copy()
        self.V = V_orig.copy()
        self.dt = original_dt / 2

        # Запускаем с уменьшенным шагом (вдвое больше шагов)
        for _ in range(test_steps * 2):
            self.rk2_step()
        result2 = self.U.copy()

        # Восстанавливаем исходный шаг
        self.dt = original_dt

        # Вычисляем разницу между решениями
        diff = np.abs(result1 - result2).max()
        print(f"Максимальная разница между решениями при dt и dt/2: {diff:.2e}")

        if diff < 1e-3:
            print("Тест на сходимость пройден: разница мала")
        else:
            print("ПРЕДУПРЕЖДЕНИЕ: значительная разница между решениями!")
            print("Рекомендуется уменьшить шаг по времени для лучшей сходимости")

            choice = input("Хотите продолжить с текущим шагом? (y/n): ").strip().lower()
            if choice != 'y':
                print("Моделирование прервано пользователем")
                return

        # Основной цикл моделирования
        early_stop = False
        for step in tqdm(range(1, self.total_steps + 1), desc="Прогресс моделирования"):
            self.rk2_step()

            # Проверка стабилизации
            if step % 100 == 0 and self.check_stabilization():
                print(f"\nСистема стабилизировалась на шаге {step}. Досрочное завершение.")
                early_stop = True
                break

            # Вывод средних значений
            if step % 10000 == 0:
                avg_U = np.mean(self.U)
                avg_V = np.mean(self.V)
                print(f"\nШаг {step}: Среднее U = {avg_U:.4f}, Среднее V = {avg_V:.4f}")

            if step % display_interval == 0:
                if step % full_display_interval == 0:
                    self.show_state(step)

                if self.with_animation:
                    self.frame_files.append(self._save_frame_to_temp_file(step, output_dir))

                    if step % gif_save_interval == 0:
                        self.save_partial_gif(output_dir)

        if not early_stop:
            print("\nМоделирование завершено по достижении максимального числа шагов!")
        else:
            print("\nМоделирование завершено досрочно из-за стабилизации системы!")

        self.show_state(self.step_count)

        if self.with_animation:
            self.create_final_animation(output_dir)

        self.save_final_patterns(output_dir)


if __name__ == "__main__":
    # 1. Сначала спрашиваем про выбор системы
    print("\n=== Выбор системы реакция-диффузия ===")
    model = ReactionDiffusionModel(with_animation=False)  # Пока анимация отключена

    # 2. Проверка условий Тьюринга для выбранной системы
    print("\n=== Проверка условий Тьюринга ===")
    turing_ok = model.check_turing_conditions()
    if not turing_ok:
        choice = input("\nУсловия Тьюринга не выполнены. Продолжить моделирование? (y/n): ").strip().lower()
        if choice != 'y':
            print("Моделирование прервано пользователем")
            exit()

    # 3. Настройка шага по времени
    print("\n=== Настройка параметров моделирования ===")
    while True:
        dt_input = input("\nВведите шаг по времени (по умолчанию 1e-7): ").strip()
        try:
            dt = float(dt_input) if dt_input else 1e-7
            model.dt = dt

            # Проверка числа Куранта
            dt_max = (model.dx ** 2) / (4 * max(model.Du, model.Dv))
            courant = dt / dt_max
            print(f"\nПроверка устойчивости:")
            print(f"Текущий dt = {dt:.2e}, максимально допустимый dt = {dt_max:.2e}")
            print(f"Число Куранта = {courant:.4f} (рекомендуется < 0.5)")

            if courant < 0.5:
                print("Условие устойчивости выполняется!")
                break
            else:
                print("\nВНИМАНИЕ: Число Куранта >= 0.5, возможна неустойчивость!")
                choice = input("Хотите ввести другой шаг по времени? (y/n): ").strip().lower()
                if choice != 'y':
                    print("Продолжаем с выбранным шагом (риск неустойчивости!)")
                    break
        except ValueError:
            print("Ошибка: введите число. Например, 1e-6 или 0.000001")

    # 4. Настройка визуализации
    print("\n=== Настройка визуализации ===")
    animate_choice = input("\nСоздавать анимацию процесса? (y/n): ").strip().lower()
    model.with_animation = animate_choice == 'y'

    # Настройки частоты вывода
    display_interval = 1000
    gif_save_interval = 50000
    full_display_interval = 10000

    print(f"\nИтоговые параметры:")
    print(f"- Выбранная система: {model.system_type}")
    print(f"- Шаг по времени: {model.dt:.2e}")
    print(f"- Создание анимации: {'включено' if model.with_animation else 'отключено'}")

    # Запуск моделирования
    input("\nНажмите Enter для начала моделирования...")
    model.run_simulation(
        output_dir="results",
        display_interval=display_interval,
        gif_save_interval=gif_save_interval,
        full_display_interval=full_display_interval
    )